In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'test.csv', 'train.csv']


In [84]:
import pandas as pd
import string, re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

In [189]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Embedding, Dropout
from keras import metrics
from keras.utils import to_categorical

In [126]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [12]:
train_raw_df = pd.read_csv("input/train.csv")
test_df = pd.read_csv("input/test.csv")
train_n = len(train_df)
test_n = len(test_df)
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (130612, 3)
Test shape :  (56370, 2)


In [135]:
train_df = train_raw_df.sample(frac=0.05)
print("Train shape : ",train_df.shape)
train_df.head(1)

Train shape :  (65306, 3)


,qid,question_text,target
737723,907a93d2f27d74b9a224,When did London lose its manners?,1


In [136]:
lemmatizer = WordNetLemmatizer()
stopset = set(stopwords.words('english'))

In [137]:
def process(txt):
    tokens = []
    for token in wordpunct_tokenize(txt):
        if token.isdigit():
            continue
        if all(char in string.punctuation for char in token):
            continue
        
        token = token.lower()
        token = token.strip()  # Strip whitespace and other punctuations
        token = token.strip('_')  # remove _ if any
        token = token.strip('*')
        if token in stopset:
            continue
        tokens.append(token)
        lemmatizer.lemmatize(token)
    return tokens

In [155]:
train_df['p_txt'] = train_df['question_text'].apply(process)
test_df['p_txt'] = test_df['question_text'].apply(process)

In [139]:
train_df.head(1)

,qid,question_text,target,p_txt
737723,907a93d2f27d74b9a224,When did London lose its manners?,1,"[london, lose, manners]"


In [156]:
train_df['p_txtcn'] = train_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))
test_df['p_txtcn'] = test_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))

In [141]:
no_features = 1000
tfidf_vectorizer = TfidfVectorizer(max_features=no_features)

In [142]:
def vectorize(txt, vectorizer):
    X = vectorizer.fit_transform(txt)
    feature_names = vectorizer.get_feature_names()
    return X, feature_names

In [143]:
X_tf, tf_feature_names = vectorize(train_df['p_txtcn'], tfidf_vectorizer)

In [144]:
no_topics = 10
num_iter = 2

In [145]:
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=num_iter, learning_method='online', learning_offset=50.,random_state=9, evaluate_every=100).fit(X_tf)

In [146]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_features = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            if topic[i] >  1.0:
                top_features.append((feature_names[i], topic[i]))
            else:
                continue        
        print("Topic %d:" % (topic_idx))
        print(", ".join([str(val[0])+": "+"%.2f" % val[1] for val in top_features]))

In [147]:
display_topics(lda, tf_feature_names, 50)

Topic 0:
best: 927.17, make: 557.96, life: 536.23, way: 480.90, better: 378.16, country: 287.58, would: 287.41, year: 269.22, long: 249.43, old: 248.40, say: 245.31, day: 215.46, live: 212.35, new: 201.18, years: 198.91, girl: 195.31, known: 143.50, eat: 140.83, friend: 138.39, another: 129.41, win: 127.12, found: 125.32, market: 115.66, believe: 110.35, less: 110.25, weight: 106.89, fight: 99.21, girlfriend: 94.76, die: 93.87, culture: 93.14, project: 91.62, term: 90.76, oil: 88.63, people: 88.29, modi: 84.46, talk: 83.82, skin: 82.17, case: 80.55, prevent: 78.88, modern: 78.05, rate: 76.65, exist: 76.15, control: 76.01, run: 75.63, sleep: 75.05, good: 74.83, price: 74.57, eating: 73.95, vs: 73.26, explain: 72.98
Topic 1:
become: 446.13, work: 432.94, take: 272.38, engineering: 246.62, anyone: 245.40, college: 241.32, without: 226.44, go: 224.12, learn: 203.51, getting: 193.47, get: 187.60, university: 185.89, look: 183.53, company: 179.51, like: 175.06, career: 170.12, students: 165.

In [148]:
X_lda = lda.transform(X_tf)

In [149]:
# X_lda[:30]

In [151]:
X_lda_df = pd.DataFrame(X_lda)
X_lda_df["idxmax"] = X_lda_df.idxmax(axis=1)
X_lda_df.head(2)

,0,1,2,3,4,5,6,7,8,9,idxmax
0,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,5
1,0.029061,0.503562,0.029061,0.029061,0.029061,0.029061,0.133664,0.029061,0.029061,0.159347,1


In [171]:
y_true = train_df["target"]
y_lda = X_lda_df["idxmax"]

In [153]:
print(confusion_matrix(y_true, y_lda)[:2])

[[9865 6852 5490 5461 6203 4329 4459 6899 5707 6023]
 [ 478  316  434  265  314  365  525  238  321  762]]


#### Trying Deep Learning with GloVe

In [229]:
def fit_get_sequences(df, vocab_size):
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(df['p_txt'])
    sequences = tokenizer.texts_to_sequences(df['p_txt'])
    return sequences, tokenizer, vocab_size

In [230]:
def get_sequences(df, tokenizer):
    return tokenizer.texts_to_sequences(df['p_txt'])

In [248]:
vocab_size = 1000
sequences, tokenizer, vocab_size = fit_get_sequences(train_df, vocab_size)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
avg = sum(map(len, sequences)) / len(sequences)
std = np.sqrt(sum(map(lambda x: (len(x) - avg) ** 2, sequences)) / len(sequences))
print("Tokens avg {} and std {}".format(avg, std))

max_length = 100
X = pad_sequences(sequences, maxlen=max_length)
X_test = pad_sequences(get_sequences(test_df, tokenizer), maxlen=max_length)

y = to_categorical(np.asarray(train_df['target']))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

Found 40712 unique tokens.
Tokens avg 3.3403515756592044 and std 2.4788018235639098


In [249]:
print('Shape of data:', X.shape)
print('Shape of labels:', y.shape)

Shape of data: (65306, 100)
Shape of labels: (65306, 2)


In [251]:
def generate_embeddings(vocab_size, word_index):
    EMBEDDING_FILE = "/Users/saurabh/Downloads/glove.6B/" + "glove.6B.100d.txt"
    embeddings_index = {}
    f = open(os.path.join(EMBEDDING_FILE))
    # In the dataset, each line represents a new word embedding
    # The line starts with the word and the embedding values follow
    for line in f:
        values = line.split()
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding
    f.close()
    all_embs = np.stack(embeddings_index.values())
    emb_mean = all_embs.mean()
    emb_std = all_embs.std()
    print(emb_mean, emb_std)
    embedding_dim = 100
    nb_words = min(vocab_size, len(word_index))  # How many words are there actually
    # Create a random matrix with the same mean and std as the embeddings
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_dim))
    # The vectors need to be in the same position as their index.
    # Meaning a word with token 1 needs to be in the second row (rows start with zero) and so on
    # Loop over all words in the word index
    for word, i in word_index.items():
        # If we are above the amount of words we want to use we do nothing
        if i >= vocab_size:
            break
        # Get the embedding vector for the word
        embedding_vector = embeddings_index.get(word)
        # If there is an embedding vector, put it in the embedding matrix
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_dim, embedding_matrix    

In [252]:
embedding_dim, embedding_matrix = generate_embeddings(vocab_size, word_index)

0.004451992 0.4081574


In [253]:
def generate_model(embedding_dim, embedding_matrix, max_length, vocab_size):
    model = Sequential()
    # model.add(Embedding(vocab_size, 100, input_length=max_length))
    model.add(
        Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
    model.add(LSTM(32, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=[metrics.mae, metrics.categorical_accuracy])
    return model

In [254]:
model = generate_model(embedding_dim, embedding_matrix, max_length, vocab_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 100)          100000    
_________________________________________________________________
lstm_11 (LSTM)               (None, 100, 32)           17024     
_________________________________________________________________
dropout_11 (Dropout)         (None, 100, 32)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 66        
_________________________________________________________________
activation_6 (Activation)    (None, 2)                 0         
Total para

In [255]:
batch_size = 1000
model.fit(X_train, y_train, batch_size=batch_size, epochs=5, validation_split=0.1)

Train on 47019 samples, validate on 5225 samples
Epoch 1/5
47019/47019 [==============================] - 50s 1ms/step - loss: 0.2718 - mean_absolute_error: 0.1563 - categorical_accuracy: 0.9277 - val_loss: 0.2262 - val_mean_absolute_error: 0.1217 - val_categorical_accuracy: 0.9361
Epoch 2/5
47019/47019 [==============================] - 50s 1ms/step - loss: 0.2163 - mean_absolute_error: 0.1135 - categorical_accuracy: 0.9390 - val_loss: 0.2140 - val_mean_absolute_error: 0.1051 - val_categorical_accuracy: 0.9361
Epoch 3/5
47019/47019 [==============================] - 52s 1ms/step - loss: 0.1985 - mean_absolute_error: 0.1049 - categorical_accuracy: 0.9394 - val_loss: 0.1902 - val_mean_absolute_error: 0.0908 - val_categorical_accuracy: 0.9378
Epoch 4/5
47019/47019 [==============================] - 48s 1ms/step - loss: 0.1818 - mean_absolute_error: 0.0967 - categorical_accuracy: 0.9405 - val_loss: 0.1785 - val_mean_absolute_error: 0.0900 - val_categorical_accuracy: 0.9391
Epoch 5/5
47019

In [256]:
def c_matrix(y_true, y_pred, num_classes=2):
    cm = confusion_matrix(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    print(cm)
    cm_np = np.asarray(cm)
    TP = np.diag(cm_np)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    TN = []
    for i in range(num_classes):
        temp = np.delete(cm, i, 0)
        temp = np.delete(temp, i, 1)
        TN.append(sum(sum(temp)))
    prec = TP / (TP + FP)
    rec = TP / (TP + FN)
    acc = (TP + TN) / (TP + FP + TN + FN)
    f1 = 2 * prec * rec / (prec + rec)

    print("accuracy", acc)
    print("precision", prec)
    print("recall", rec)
    print("f1", f1)
    return prec, rec, acc, f1

In [257]:
y_pred = model.predict(X_test)

c_matrix(y_test, y_pred, num_classes=2)

[[12149    96]
 [  709   108]]
accuracy [0.93837085 0.93837085]
precision [0.94485923 0.52941176]
recall [0.99216007 0.13219094]
f1 [0.96793212 0.2115573 ]


(array([0.94485923, 0.52941176]),
 array([0.99216007, 0.13219094]),
 array([0.93837085, 0.93837085]),
 array([0.96793212, 0.2115573 ]))